# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv
from prettytable import PrettyTable

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [4]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. 
The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [5]:
# makeing a connection to a Cassandra instance your local machine 


from cassandra.cluster import Cluster
cluster = Cluster()

session = cluster.connect()

#### Create Keyspace

In [6]:
# Create a Keyspace 
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS udacity 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)

#### Set Keyspace

In [7]:
# Set KEYSPACE to the keyspace specified above
try:
    session.set_keyspace('udacity')
except Exception as e:
    print(e)

### Now we need to create tables to run the following queries. with Apache Cassandra you model the database tables on the queries you want to run.

## queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




In [8]:
## Query 1:  Give me the artist, song title and song's length in the music app history that was heard during \
## sessionId = 338, and itemInSession = 4

query = "CREATE TABLE IF NOT EXISTS song_info_by_session "
query = query + "(sessionId int, itemInSession int, artist text, \
                  song_title text, length float, \
                  PRIMARY KEY (sessionId, itemInSession))"
try:
    session.execute(query)
except Exception as e:
    print(e)         

In [9]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO song_info_by_session (sessionId, itemInSession, artist, \
                  song_title, length)"
        query = query + " VALUES (%s, %s, %s, %s, %s)"
        length = float(line[5])
        sessionId = int(line[8])
        itemInSession = int(line[3])
        session.execute(query, (sessionId, itemInSession, line[0], line[9], length))

#### SELECT to verify that the data have been inserted into the table when trying to get  artist name, song title and the length using the partation key sessionId and itemInSession

In [10]:
## verifying the data was entered into the table
query = "SELECT artist, song_title, length from song_info_by_session WHERE sessionId = 338 and itemInSession = 4"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
x = PrettyTable()
x.field_names = ["artist", "song title", "length"]
for row in rows:
    x.add_row([row.artist, row.song_title, row.length])
print(x)

+-----------+---------------------------------+--------------------+
|   artist  |            song title           |       length       |
+-----------+---------------------------------+--------------------+
| Faithless | Music Matters (Mark Knight Dub) | 495.30731201171875 |
+-----------+---------------------------------+--------------------+


In [11]:
## Query 2: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name)\
## for userid = 10, sessionid = 182
## adding artist column as a CLUSTERING COLUMN to get song with the same title for different artists
query = "CREATE TABLE IF NOT EXISTS song_info_by_itemInSession "
query = query + "(userId int, sessionId int, itemInSession int, artist text, song_title text, \
                first_name text, last_name text,  \
                  PRIMARY KEY ((userId, sessionId), itemInSession, first_name, last_name))"
try:
    session.execute(query)
except Exception as e:
    print(e)
                    

In [12]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO song_info_by_itemInSession (userId, sessionId,  itemInSession, artist, song_title,\
                first_name, last_name)"
        query = query + " VALUES (%s, %s, %s, %s, %s, %s, %s)"
        userId = int(line[10])
        sessionId = int(line[8])
        itemInSession = int(line[3])
        session.execute(query, (userId, sessionId, itemInSession, line[0], line[9], line[1], line[4]))

#### SELECT to verify that the data have been inserted into the table when trying to get  artist name and song title  using the partation key userId and sessionId

In [13]:
## verifying the data was entered into the table
query = "SELECT artist, song_title from song_info_by_itemInSession WHERE userId = 10 AND sessionId = 182"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
x = PrettyTable()
x.field_names = ["artist", "song title"]
for row in rows:
    x.add_row([row.artist, row.song_title])
print(x)

+-------------------+------------------------------------------------------+
|       artist      |                      song title                      |
+-------------------+------------------------------------------------------+
|  Down To The Bone |                  Keep On Keepin' On                  |
|    Three Drives   |                     Greece 2000                      |
| Sebastien Tellier |                      Kilometer                       |
|   Lonnie Gordon   | Catch You Baby (Steve Pitron & Max Sanna Radio Edit) |
+-------------------+------------------------------------------------------+


In [14]:
## Query 3: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

query = "CREATE TABLE IF NOT EXISTS user_data_listened_to_a_song "
query = query + "(song_title text, userId int, first_name text, last_name text, \
                  PRIMARY KEY (song_title, userId))"
try:
    session.execute(query)
except Exception as e:
    print(e)

                    

In [15]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO user_data_listened_to_a_song (song_title, userId, first_name, last_name )"
        query = query + " VALUES (%s, %s, %s, %s)"
        userId = int(line[10])
        session.execute(query, (line[9], userId, line[1], line[4]))

#### SELECT to verify that the data have been inserted into the table when trying to get  the first and last name of a user  using the partation key song_title

In [16]:
## verifying the data was entered into the table
query = "SELECT first_name, last_name from user_data_listened_to_a_song WHERE song_title = 'All Hands Against His Own'"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
x = PrettyTable()
x.field_names = ["first name", "last name "]
for row in rows:
    x.add_row([row.first_name, row.last_name])
print(x)

+------------+------------+
| first name | last name  |
+------------+------------+
| Jacqueline |   Lynch    |
|   Tegan    |   Levine   |
|    Sara    |  Johnson   |
+------------+------------+


### Drop the tables before closing out the sessions

In [17]:
## Droping the table before closing out the sessions

In [18]:
query = "drop table song_info_by_session"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
query = "drop table song_info_by_itemInSession"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
query = "drop table user_data_listened_to_a_song"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

### Close the session and cluster connection¶

In [19]:
session.shutdown()
cluster.shutdown()